# Translate args_performance.txt to table

In [102]:
dataset = "gowalla3k_5k"
outfile = open('tb_args_performance.csv', 'w')

def str2intlist(s):
    s = s.strip('[').strip(']').split(',')
    l = [int(i) for i in s]
    return l

unprint_args = ['pretrain', 'epoch', 'weights_path', 'regs', 'alg_type', 'gpu_id',  'adj_type',
                'embed_size', 'Ks', 'data_path',  'test_flag', 'report_loop', 'mess_dropout',
                'verbose', 'dataset', 'model_type', 'report', 'lr', 'save_flag', 'proj_path',
                'keep_prob', 'save_print_flag']

with open('args_performance_' + dataset + '.txt', 'r') as infile:
    odd_flag = 0
    header_flag = 1
    field_name_dic = {}
    field_name_list_keys = []
    list_ks = []
    print_args = []
    
    for l in infile.readlines():
        l = l.strip('\n')
        odd_flag = 1 - odd_flag
        
        if odd_flag:
            # Namespace(Ks='[10]', adj_type='norm', ...)
            l = l.strip('Namespace').strip('(').strip(')').replace("'","")
            for field in l.split(', '):
                field = field.split('=')
                if field.__len__() > 1:
                    field_name_dic[field[0]] = field[1]
                else:
                    field_name_dic[field[0]] = ''
                
            if header_flag:
                list_ks = str2intlist(field_name_dic['Ks'])
                headline = 'idx,t_train'  # headline = 'idx,t_train,Recall,Precision,HR,NDCG'
                for fn in ['Recall','Precision','HR','NDCG']:
                    for k in list_ks:
                        headline += ',' + fn + '@' + str(k)
                field_name_list_keys = field_name_dic.keys()
                
                print_args = list(set(field_name_list_keys) - set(unprint_args))
#                 for fn in field_name_list_keys:
                for fn in print_args:
                    headline += ',' + fn
                headline = headline.replace(',','\t')
                print(headline, file=outfile)
                header_flag = 0
        else:
#             outline = l.replace(',','\t')
#             list_ks = field_name_dic['Ks']
            l = l.split('\t')
            outline = l[0] + '\t' + l[1]
            for li in l[2:]:
                li = li.strip('[').strip(']').replace("'",'').replace(', ','\t')
                outline += '\t' + li
#             for k in field_name_list_keys:
            for k in print_args:
                outline += '\t' + field_name_dic[k]
            print(outline, file=outfile)

outfile.close()


In [70]:
print(field_name_list_keys)

dict_keys(['node_dropout', 'pretrain', 'epoch', 'weights_path', 'regs', 'alg_type', 'gpu_id', 'shuffle_rate', 'adj_type', 'embed_size', 'Ks', 'data_path', 'num_neg', 'test_flag', 'report_loop', 'mess_dropout', 'verbose', 'dataset', 'model_type', 'report', 'lr', 'save_flag', 'proj_path', 'layer_size', 'node_dropout_flag', 'batch_size', 'social', 'keep_prob', 'save_print_flag'])


# Generate .sh batch script

In [90]:
# run the args_performance.txt processing cell first to get field_name_dic

# select dataset
field_name_dic['dataset'] = 'gowalla3k_5k'
# field_name_dic['dataset'] = 'gowalla8k'
# field_name_dic['dataset'] = 'gowalla_sample_ngcf'

# choose times of experiment
N_times = 3

# variants
list_batch_size = ['256', '1024']
# list_layer_size = ['[64,64]']
# list_lr = ['0.0001']
list_num_neg = ['10', '8', '4', '2', '1']
list_social = ['1', '0']
list_shuffle_rate = ['0.0', '0.2', '0.4', '0.6', '0.8', '1.0']
# list_social_shuffle = []
# for s in list_social:
#     if int(s) == 0:
#         list_social_shuffle.append('0')
#     else:
#         for sr in list_shuffle_rate:

selected_field_name = ['batch_size', 'num_neg', 'social', 'shuffle_rate']
unselected_field_name = list(set(field_name_dic.keys()) - set(selected_field_name))
if 'weights_path' in unselected_field_name:
    unselected_field_name.remove('weights_path')
    unselected_field_name.remove('proj_path')

with open('batch_' + field_name_dic['dataset'] + '.sh', 'w') as f:
    for lp in range(N_times):
        for batch_size in list_batch_size:
            for num_neg in list_num_neg:
                for social in list_social:
                    if int(social) == 0:
                        cmdline = "python NGCF.py --shuffle_rate 0. --batch_size " + batch_size + " --num_neg " + num_neg \
                        + " --social " + social
                        for fn in unselected_field_name:
                            cmdline += " --" + fn + " " + field_name_dic[fn]
                        print(cmdline, file=f)
                    else:
                        for shuffle_rate in list_shuffle_rate:
                            cmdline = "python NGCF.py --shuffle_rate " + shuffle_rate + " --batch_size " + batch_size \
                            + " --num_neg " + num_neg + " --social " + social
                            for fn in unselected_field_name:
                                cmdline += " --" + fn + " " + field_name_dic[fn]
                            print(cmdline, file=f)
                            

    

In [86]:
field_name_dic

{'Ks': '[10,20,40,100,1]',
 'adj_type': 'norm',
 'alg_type': 'ngcf',
 'batch_size': '256',
 'data_path': '../Data/',
 'dataset': 'gowalla8k',
 'embed_size': '64',
 'epoch': '400',
 'gpu_id': '0',
 'keep_prob': '0.5',
 'layer_size': '[64,64]',
 'lr': '0.0001',
 'mess_dropout': '[0.1,0.1,0.1]',
 'model_type': 'ngcf',
 'node_dropout': '[0.1]',
 'node_dropout_flag': '0',
 'num_neg': '10',
 'pretrain': '0',
 'proj_path': '',
 'regs': '[1e-5]',
 'report': '0',
 'report_loop': '10',
 'save_flag': '1',
 'save_print_flag': '1',
 'shuffle_rate': '0.0',
 'social': '1',
 'test_flag': 'part',
 'verbose': '1',
 'weights_path': ''}

In [96]:
# run the args_performance.txt processing cell first to get field_name_dic

for i in range(6,10):
    # select dataset
    field_name_dic['dataset'] = 'drop_0.' + str(i)
    # field_name_dic['dataset'] = 'gowalla8k'
    # field_name_dic['dataset'] = 'gowalla_sample_ngcf'

    # choose times of experiment
    N_times = 3

    # variants
    list_batch_size = ['256']
    # list_layer_size = ['[64,64]']
    # list_lr = ['0.0001']
    list_num_neg = ['10']
    list_social = ['1', '0']
    list_shuffle_rate = ['0.0']
    # list_social_shuffle = []
    # for s in list_social:
    #     if int(s) == 0:
    #         list_social_shuffle.append('0')
    #     else:
    #         for sr in list_shuffle_rate:

    selected_field_name = ['batch_size', 'num_neg', 'social', 'shuffle_rate']
    unselected_field_name = list(set(field_name_dic.keys()) - set(selected_field_name))
    if 'weights_path' in unselected_field_name:
        unselected_field_name.remove('weights_path')
        unselected_field_name.remove('proj_path')

    with open('batch_drops.sh', 'a') as f:
        for lp in range(N_times):
            for batch_size in list_batch_size:
                for num_neg in list_num_neg:
                    for social in list_social:
                        if int(social) == 0:
                            cmdline = "python NGCF.py --shuffle_rate 0. --batch_size " + batch_size + " --num_neg " + num_neg \
                            + " --social " + social
                            for fn in unselected_field_name:
                                cmdline += " --" + fn + " " + field_name_dic[fn]
                            print(cmdline, file=f)
                        else:
                            for shuffle_rate in list_shuffle_rate:
                                cmdline = "python NGCF.py --shuffle_rate " + shuffle_rate + " --batch_size " + batch_size \
                                + " --num_neg " + num_neg + " --social " + social
                                for fn in unselected_field_name:
                                    cmdline += " --" + fn + " " + field_name_dic[fn]
                                print(cmdline, file=f)




In [103]:
0.09483 / 0.08985 - 1

0.05542570951585968

In [104]:
0.09483 - 0.08985

0.004979999999999998

In [1]:
.09483 / .0841 - 1

0.12758620689655187

In [2]:
drop_sngcf = [0.07388,0.07688,0.07849,0.08218,0.09483]
drop_ngcf = [0.063,0.06804,0.07191,0.08038,0.0841]
drop_bpr = [0.05512,0.06156,0.06934,0.07365,0.07288]

In [3]:
line = ""
for d in drop_sngcf:
    line += "\t" + str(d)
line += "\n"
for d in drop_ngcf:
    line += "\t" + str(d)
line += "\n"
for d in drop_bpr:
    line += "\t" + str(d)
print(line)

	0.07388	0.07688	0.07849	0.08218	0.09483
	0.063	0.06804	0.07191	0.08038	0.0841
	0.05512	0.06156	0.06934	0.07365	0.07288
